# Burgers' Equation

The inviscid Burgers' equation in one space dimension is
$$
\partial_t u + \partial_x \frac{u^2}{2} = 0.
$$
It is a scalar conservation law with convex flux $f(u) = \frac{u^2}{2}$.

## Riemann Problem

In [ ]:
using HyperbolicDiffEq
using Plots; pyplot()

# create a Riemann problem with left value 0 and right value 1
prob1 = RiemannProblem(Burgers(), 0., 1.)
sol1  = solve(prob1)
plot1 = plot(sol1)

# create a second Riemann problem at x₀=1
prob2 = RiemannProblem(Burgers(), 1., -1., 1.)
sol2  = solve(prob2)
plot2 = plot(sol2)

plot(plot1, plot2)

### Consecutive Riemann Problems

In [ ]:
prob3 = RiemannProblem(Burgers(), -1., -2., 2)

prob = prob1 * prob2
prob = prob * prob3
sol = solve(prob)
plot(sol)

# First Order Finite Volume Methods

In [ ]:
using Revise
using HyperbolicDiffEq
using OrdinaryDiffEq, DiffEqCallbacks
using LaTeXStrings, Plots; pyplot()

xmin, xmax = -2., 2.
N = 10^3
usethreads = true

balance_law = Burgers()
numflux = GodunovFlux() # GodunovFlux, LocalLaxFriedrichsFlux

# set up a periodic problem
u_ana = solve(RiemannProblem(balance_law, 1.e-4, 1., -0.5) * RiemannProblem(balance_law, 1., 1.e-4, 0.5))
tspan = (0., 1.)
u₀func(x) = u_ana(tspan[1], x)

# create an ODE
mesh = UniformPeriodicMesh1D(xmin, xmax, N)
fv = FirstOrderFV(balance_law, mesh, numflux, usethreads)
ode = semidiscretise(fv, u₀func, tspan)

# solve the ODE sing either an adaptive time step (CFL) or a fixed one
#cb = StepsizeLimiter((t,u)->max_dt(t,u,fv), safety_factor=1, max_step=true)
#sol = solve(ode, OrdinaryDiffEq.Euler(), dt=1, callback=cb, save_everystep=false)
sol = solve(ode, OrdinaryDiffEq.Euler(), dt=max_dt(tspan[1], ode.u0, fv), save_everystep=false)

plot(xguide=L"x", yguide=L"u")
plot!(evaluate_coefficients(sol[1], mesh), label=L"u_0")
plot!(evaluate_coefficients(sol[end], mesh), label=L"u_\mathrm{num}")
x = range(xmin, xmax, length=10^3)
plot!(x, u_ana.(tspan[end], x), label=L"u_\mathrm{ana}", legend=true)